In [1]:
import pandas as pd
import numpy as np
import datetime

In [2]:
names = ['Course ID','Subject','Catalog','Descr','Status','Eff Date','Consent','Grading','Min Units','Max Units','Repeatable','Allowd Unt','Allow Comp','Allow Mult','Campus','Acad Org','Career','Long Title','Description']

In [3]:
courses = pd.read_excel('Course Table.xls',names = names )
courses = courses.iloc[1:]

WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero


In [4]:
columns= ['Term','Career','Acad Org','Subject','Catalog',
          'Descr','Class Nbr','Section','Room','Days','Start Time','Mtg End','Class Stat','Enrl Stat','Enroll Cap',
          'Current enrollment','Session','Sort days','Location','Primary Instructor Name','Min Units','Max Units',
          'Start Date','End Date','Acad Group','Campus']
schedule = pd.read_excel('Schedule Table.xls',names=columns)
schedule = schedule.iloc[1:]

In [5]:
schedule['Term'] = schedule.Term.astype(str).str[2:].astype(int)
schedule['Year'] = schedule['Start Date'].dt.year

In [6]:
schedule = schedule[['Year','Term','Subject','Catalog',
          'Descr','Class Nbr','Primary Instructor Name','Max Units','Start Date','End Date']]

In [7]:
def term(term_number):
    if term_number == 0:
        return 'Fall'
    elif term_number == 20:
         return 'Winter'
    elif term_number == 40:
        return 'Spring'
    elif term_number == 60:
        return 'Summer'
schedule['Term'] = schedule['Term'].apply(term)

In [8]:
courses = courses[['Course ID','Subject','Catalog','Long Title','Description']]
courses.rename(columns = {'Descr':'Description'},inplace=True)

In [9]:
unique_courses = courses.drop_duplicates(subset=['Subject','Catalog'], keep='first')

In [305]:
unique_courses

,Course ID,Subject,Catalog,Long Title,Description
1,063018,ACTG,300,Financial Accounting,"Introduces the roles, concepts, principles, le..."
2,050122,ACTG,301,Actg for Business Decisions,NaN
3,063045,ACTG,302,Managerial Accounting,Focused study of issues relating to the measur...
4,010049,ACTG,303,Corporate Financial Reporting,NaN
6,050123,ACTG,305,Financial Statement Analy,NaN
7,010051,ACTG,307,Tax Aspects of Business Decisions,NaN
8,010052,ACTG,309,Seminar in International Accounting,Financial reporting issues facing entities con...
9,010053,ACTG,311,Management Accounting: Incentives & Performanc...,NaN
10,010054,ACTG,313,Management Accounting: Strategic Cost Management,NaN
12,062261,ACTG,317,Taxation and Personal Business Strategy,Builds a strong foundation of basic principles...


In [10]:
merge_schedule_course = schedule.merge(unique_courses,on=['Subject','Catalog'],how='left')

In [170]:
msis_courses = pd.read_excel('Curriculum.xlsx')
msis_courses = msis_courses[['Label', 'Subject', 'Course Number']]
msis_courses.rename(columns={'Course Number':'Catalog'},inplace=True)
msis_courses = msis_courses.drop_duplicates(subset=['Subject','Catalog'],keep='first')
msis_courses = msis_courses[msis_courses['Label'].notna()]
msis_courses['Subject'] = msis_courses['Subject'].astype(str)
msis_courses['Catalog'] = msis_courses['Catalog'].astype(str)
msis_courses['Catalog'] = msis_courses['Catalog'].str.strip()
msis_courses['Catalog'] = msis_courses['Catalog'].str.split(',')

In [171]:
# Making each course number into new row
s = msis_courses.apply(lambda x: pd.Series(x['Catalog']),axis=1).stack().reset_index(level=1, drop=True)
s.name = 'Catalog'
msis_courses = msis_courses.drop('Catalog', axis=1).join(s)

In [174]:
data = merge_schedule_course.merge(msis_courses,on=['Subject','Catalog'],how='left')
msis_data = data[data['Label'].isin(msis_courses['Label'])]

In [178]:
data_to_concat = data[data.Subject == 'MSIS']

In [180]:
all_data = pd.concat([msis_data,data_to_concat])

In [183]:
all_data

,Year,Term,Subject,Catalog,Descr,Class Nbr,Primary Instructor Name,Max Units,Start Date,End Date,Course ID,Long Title,Description,Label
69,2019,Fall,MGMT,2501,Bldg & Lding Hi-Perf Teams/Org,92420,"Gordon,Francine Ethel",3,2019-09-23 00:00:00,2019-12-07 00:00:00,064512,Building and Leading High-Performance Teams an...,Leaders are successful when they build teams a...,Business Core
119,2019,Fall,MSIS,2601,Object-Oriented Analysis & Prg,92471,"Ghiassi,Manoochehr",4,2019-09-23 00:00:00,2019-12-07 00:00:00,062434,Object-Oriented Software Design Analysis and P...,Provide students with an overview of object-or...,MSIS Core
120,2019,Fall,MSIS,2601,Object-Oriented Analysis & Prg,92472,"Ghiassi,Manoochehr",4,2019-09-21 00:00:00,2019-12-07 00:00:00,062434,Object-Oriented Software Design Analysis and P...,Provide students with an overview of object-or...,MSIS Core
121,2019,Fall,MSIS,2607,Data Analytics - Python,92424,"Tan,Tzu-Chin",4,2019-09-23 00:00:00,2019-12-07 00:00:00,065078,Data Analytics - Python,Data analytics involves the application of sci...,MSIS Core
122,2019,Fall,MSIS,2621,Bus Intelligence & Data Wrhsng,92425,"Chavali,Sreeram Kishore",4,2019-09-23 00:00:00,2019-12-07 00:00:00,062620,Business Intelligence & Data Warehousing,Relevant information is essential to any busin...,MSIS Elective
123,2019,Fall,MSIS,2627,Big Data Modeling & Analytics,92426,"Parsian,Mahmoud",4,2019-09-23 00:00:00,2019-12-07 00:00:00,064986,Big Data Modeling and Analytics,"Today, more than 80% of useful data is unstruc...",MSIS Elective
124,2019,Fall,MSIS,2627,Big Data Modeling & Analytics,92427,"Parsian,Mahmoud",4,2019-09-23 00:00:00,2019-12-07 00:00:00,064986,Big Data Modeling and Analytics,"Today, more than 80% of useful data is unstruc...",MSIS Elective
125,2019,Fall,MSIS,2628,Applied Cloud Compting,92428,"Wu,Feihong",2,2019-09-23 00:00:00,2019-12-07 00:00:00,064987,Applied Cloud Computing,Computing is migrating to cloud. In this cours...,MSIS Elective
126,2019,Fall,MSIS,2631,Machine Learning,92431,"Wadhwa,Sudhir K.",4,2019-09-23 00:00:00,2019-12-07 00:00:00,065488,Machine Learning,This course introduces participants to quantit...,MSIS Elective
127,2019,Fall,MSIS,2641,Info Tech: Ethics & Pub Policy,92436,"Seidler,Simon",3,2019-09-23 00:00:00,2019-12-07 00:00:00,062678,Information Technology: Ethics and Public Policy,Introduces the normative principles for ethica...,MSIS Elective


In [125]:
pre_req = pd.read_excel('prereq.xls')
pre_req.rename(columns={'Nbr':'Catalog'},inplace=True)
new_pre = pre_req.drop_duplicates(subset=['Subject','Catalog'],keep='first')
new_pre = new_pre[['Subject','Catalog','Requisite']]
new_pre.Subject = new_pre.Subject.astype(str)
new_pre.Catalog = new_pre.Catalog.astype(str)

In [126]:
# new_pre.Requisite = new_pre.Requisite.str.replace('OR','|')

In [369]:
data_with_prereq = all_data.merge(new_pre,on=['Subject','Catalog'],how='left')

In [370]:
# data_with_prereq.Requisite = data_with_prereq.Requisite.str.replace('OR','|')
# data_with_prereq.Requisite = data_with_prereq.Requisite.str.replace('MSINFSYSMJ Academic Plan ONLY','')
# data_with_prereq.Requisite = data_with_prereq.Requisite.str.replace('MSINFSYSMJ ONLY','')
# data_with_prereq.Requisite = data_with_prereq.Requisite.str.replace('MSFNCEMJ','')
# data_with_prereq.Requisite = data_with_prereq.Requisite.str.replace('MSINFSYSMJ','')
# data_with_prereq.Requisite = data_with_prereq.Requisite.fillna('')

In [371]:
data_with_prereq

,Year,Term,Subject,Catalog,Descr,Class Nbr,Primary Instructor Name,Max Units,Start Date,End Date,Course ID,Long Title,Description,Label,Requisite
0,2019,Fall,MGMT,2501,Bldg & Lding Hi-Perf Teams/Org,92420,"Gordon,Francine Ethel",3,2019-09-23 00:00:00,2019-12-07 00:00:00,064512,Building and Leading High-Performance Teams an...,Leaders are successful when they build teams a...,Business Core,(MSINFSYSMJ OR MSFNCEMJ)
1,2019,Fall,MSIS,2601,Object-Oriented Analysis & Prg,92471,"Ghiassi,Manoochehr",4,2019-09-23 00:00:00,2019-12-07 00:00:00,062434,Object-Oriented Software Design Analysis and P...,Provide students with an overview of object-or...,MSIS Core,MSINFSYSMJ Academic Plan ONLY
2,2019,Fall,MSIS,2601,Object-Oriented Analysis & Prg,92472,"Ghiassi,Manoochehr",4,2019-09-21 00:00:00,2019-12-07 00:00:00,062434,Object-Oriented Software Design Analysis and P...,Provide students with an overview of object-or...,MSIS Core,MSINFSYSMJ Academic Plan ONLY
3,2019,Fall,MSIS,2607,Data Analytics - Python,92424,"Tan,Tzu-Chin",4,2019-09-23 00:00:00,2019-12-07 00:00:00,065078,Data Analytics - Python,Data analytics involves the application of sci...,MSIS Core,MSINFSYSMJ Academic Plan ONLY
4,2019,Fall,MSIS,2621,Bus Intelligence & Data Wrhsng,92425,"Chavali,Sreeram Kishore",4,2019-09-23 00:00:00,2019-12-07 00:00:00,062620,Business Intelligence & Data Warehousing,Relevant information is essential to any busin...,MSIS Elective,(MSIS 603 OR MSIS 2603) AND MSINFSYSMJ
5,2019,Fall,MSIS,2627,Big Data Modeling & Analytics,92426,"Parsian,Mahmoud",4,2019-09-23 00:00:00,2019-12-07 00:00:00,064986,Big Data Modeling and Analytics,"Today, more than 80% of useful data is unstruc...",MSIS Elective,(MSIS 601 OR 2601) AND (MSIS 603 OR 2603) AND ...
6,2019,Fall,MSIS,2627,Big Data Modeling & Analytics,92427,"Parsian,Mahmoud",4,2019-09-23 00:00:00,2019-12-07 00:00:00,064986,Big Data Modeling and Analytics,"Today, more than 80% of useful data is unstruc...",MSIS Elective,(MSIS 601 OR 2601) AND (MSIS 603 OR 2603) AND ...
7,2019,Fall,MSIS,2628,Applied Cloud Compting,92428,"Wu,Feihong",2,2019-09-23 00:00:00,2019-12-07 00:00:00,064987,Applied Cloud Computing,Computing is migrating to cloud. In this cours...,MSIS Elective,MSIS 2601 AND MSIS 2603 AND MSIS 2605 AND MSIN...
8,2019,Fall,MSIS,2631,Machine Learning,92431,"Wadhwa,Sudhir K.",4,2019-09-23 00:00:00,2019-12-07 00:00:00,065488,Machine Learning,This course introduces participants to quantit...,MSIS Elective,MSIS 2601 AND MSIS 2603 AND MSINFSYSMJ
9,2019,Fall,MSIS,2641,Info Tech: Ethics & Pub Policy,92436,"Seidler,Simon",3,2019-09-23 00:00:00,2019-12-07 00:00:00,062678,Information Technology: Ethics and Public Policy,Introduces the normative principles for ethica...,MSIS Elective,MSINFSYSMJ Academic Plan ONLY


In [372]:
data_with_prereq['Subject'] = data_with_prereq['Subject'].astype(str)
data_with_prereq['Catalog'] = data_with_prereq['Catalog'].astype(str)
data_with_prereq['Subject'] = data_with_prereq['Subject'].str.strip()
data_with_prereq['Catalog'] = data_with_prereq['Catalog'].str.strip()

In [373]:
data_with_prereq['Subject_Catalog'] = data_with_prereq['Subject'] + ' ' + data_with_prereq['Catalog']

In [374]:
data_with_prereq.drop(columns=['Subject','Catalog','Class Nbr','Start Date','End Date','Course ID'],inplace=True)

In [375]:
data_with_prereq['Term'] = data_with_prereq['Term'].astype(str)
data_with_prereq['Year'] = data_with_prereq['Year'].astype(str)

In [376]:
data_with_prereq['Term_Offered'] = data_with_prereq['Term'] + ' ' + data_with_prereq['Year']

In [377]:
data_with_prereq.drop(columns=['Year','Term'],inplace=True)

In [378]:
data_with_prereq.columns

Index(['Descr', 'Primary Instructor Name', 'Max Units', 'Long Title',
       'Description', 'Label', 'Requisite', 'Subject_Catalog', 'Term_Offered'],
      dtype='object')

In [391]:
data_with_prereq['Primary Instructor Name'] = data_with_prereq['Primary Instructor Name'].str.replace(',',' ')

In [392]:
data_with_prereq

,Descr,Primary Instructor Name,Long Title,Description,Label,Requisite,Subject_Catalog,Units_Term
0,Bldg & Lding Hi-Perf Teams/Org,Gordon Francine Ethel,Building and Leading High-Performance Teams an...,Leaders are successful when they build teams a...,Business Core,(MSINFSYSMJ OR MSFNCEMJ),MGMT 2501,3 Units - Fall 2019
1,Object-Oriented Analysis & Prg,Ghiassi Manoochehr,Object-Oriented Software Design Analysis and P...,Provide students with an overview of object-or...,MSIS Core,MSINFSYSMJ Academic Plan ONLY,MSIS 2601,4 Units - Fall 2019
2,Object-Oriented Analysis & Prg,Ghiassi Manoochehr,Object-Oriented Software Design Analysis and P...,Provide students with an overview of object-or...,MSIS Core,MSINFSYSMJ Academic Plan ONLY,MSIS 2601,4 Units - Fall 2019
3,Data Analytics - Python,Tan Tzu-Chin,Data Analytics - Python,Data analytics involves the application of sci...,MSIS Core,MSINFSYSMJ Academic Plan ONLY,MSIS 2607,4 Units - Fall 2019
4,Bus Intelligence & Data Wrhsng,Chavali Sreeram Kishore,Business Intelligence & Data Warehousing,Relevant information is essential to any busin...,MSIS Elective,(MSIS 603 OR MSIS 2603) AND MSINFSYSMJ,MSIS 2621,4 Units - Fall 2019
5,Big Data Modeling & Analytics,Parsian Mahmoud,Big Data Modeling and Analytics,"Today, more than 80% of useful data is unstruc...",MSIS Elective,(MSIS 601 OR 2601) AND (MSIS 603 OR 2603) AND ...,MSIS 2627,4 Units - Fall 2019
6,Big Data Modeling & Analytics,Parsian Mahmoud,Big Data Modeling and Analytics,"Today, more than 80% of useful data is unstruc...",MSIS Elective,(MSIS 601 OR 2601) AND (MSIS 603 OR 2603) AND ...,MSIS 2627,4 Units - Fall 2019
7,Applied Cloud Compting,Wu Feihong,Applied Cloud Computing,Computing is migrating to cloud. In this cours...,MSIS Elective,MSIS 2601 AND MSIS 2603 AND MSIS 2605 AND MSIN...,MSIS 2628,2 Units - Fall 2019
8,Machine Learning,Wadhwa Sudhir K.,Machine Learning,This course introduces participants to quantit...,MSIS Elective,MSIS 2601 AND MSIS 2603 AND MSINFSYSMJ,MSIS 2631,4 Units - Fall 2019
9,Info Tech: Ethics & Pub Policy,Seidler Simon,Information Technology: Ethics and Public Policy,Introduces the normative principles for ethica...,MSIS Elective,MSINFSYSMJ Academic Plan ONLY,MSIS 2641,3 Units - Fall 2019


In [379]:
data_with_prereq['Max Units'] = data_with_prereq['Max Units'].astype(str)

In [380]:
data_with_prereq['Units_Term'] = data_with_prereq['Max Units'] + ' Units - ' + data_with_prereq['Term_Offered']

In [381]:
data_with_prereq.drop(columns=['Max Units','Term_Offered'],inplace=True)

In [393]:
instructor_list  = data_with_prereq.groupby(['Subject_Catalog'])[['Primary Instructor Name']].apply(lambda x: x.values.tolist())

In [394]:
instructor_data = instructor_list.to_frame()

In [395]:
instructor_data.reset_index(inplace=True)

In [396]:
instructor_data.rename(columns={0:'Instructor_Name'},inplace=True)

In [397]:
instructor_data

,Subject_Catalog,Instructor_Name
0,ACTG 2608,"[[Teruel Frank], [Teruel Frank]]"
1,MGMT 2501,"[[Gordon Francine Ethel], [Zeitlin Jo-Ellen Po..."
2,MSIS 2403,[[Chavali Sreeram Kishore]]
3,MSIS 2406,"[[Chung Sunghun], [Chung Sunghun], [Chung Sung..."
4,MSIS 2407,"[[Li Tao], [Tan Tzu-Chin], [Li Tao], [Tan Tzu-..."
5,MSIS 2428,[[Parsian Mahmoud]]
6,MSIS 2429,[[Khan Arshad]]
7,MSIS 2431,"[[Parsian Mahmoud], [Parsian Mahmoud]]"
8,MSIS 2439,[[Khan Arshad]]
9,MSIS 2440,[[Goldenberg Charles Bruce]]


In [399]:
instructor_data.Instructor_Name = instructor_data.Instructor_Name.apply(np.unique)

In [400]:
instructor_data

,Subject_Catalog,Instructor_Name
0,ACTG 2608,[Teruel Frank]
1,MGMT 2501,"[Gordon Francine Ethel, Griffith Terri L, Zeit..."
2,MSIS 2403,[Chavali Sreeram Kishore]
3,MSIS 2406,[Chung Sunghun]
4,MSIS 2407,"[Li Tao, Tan Tzu-Chin]"
5,MSIS 2428,[Parsian Mahmoud]
6,MSIS 2429,[Khan Arshad]
7,MSIS 2431,[Parsian Mahmoud]
8,MSIS 2439,[Khan Arshad]
9,MSIS 2440,[Goldenberg Charles Bruce]


In [401]:
term_list  = data_with_prereq.groupby(['Subject_Catalog'])[['Units_Term']].apply(lambda x: x.values.tolist())
term_list = term_list.to_frame()
term_list.reset_index(inplace=True)
term_list.rename(columns={0:'Term_Offered'},inplace=True)
term_list.Term_Offered = term_list.Term_Offered.apply(np.unique)

In [402]:
instructor_term = instructor_data.merge(term_list, on = ['Subject_Catalog'],how='left')

In [403]:
data_with_prereq = data_with_prereq[['Subject_Catalog','Descr','Long Title','Description','Label','Requisite']]

In [404]:
unique_data = data_with_prereq.drop_duplicates(subset=['Subject_Catalog'],keep='first')

In [504]:
final_data = instructor_term.merge(unique_data,on = ['Subject_Catalog'],how='left')

In [505]:
# final_data.Requisite = final_data.Requisite.str.replace('OR','|')

In [506]:
final_data.Requisite = final_data.Requisite.fillna('')

In [507]:
final_data.Requisite = final_data.Requisite.str.upper()
final_data.Requisite = final_data.Requisite.str.replace('(MSINFSYSMJ OR MSFNCEMJ)','')
final_data.Requisite = final_data.Requisite.str.replace('(MSFNCEMJ OR MSENTREPMJ)','')
final_data.Requisite = final_data.Requisite.str.replace('MSBUSAMJ ACADEMIC','')
final_data.Requisite = final_data.Requisite.str.replace('Academic PLAN ONLY','')
final_data.Requisite = final_data.Requisite.str.replace('MSBUSAMJ ACADEMIC PLAN ','')
final_data.Requisite = final_data.Requisite.str.replace('ONLY','')
final_data.Requisite = final_data.Requisite.str.replace('MSINFSYSMJ','')
final_data.Requisite = final_data.Requisite.str.replace('(','')
final_data.Requisite = final_data.Requisite.str.replace(')','')
final_data.Requisite = final_data.Requisite.str.replace('ACADEMIC PLAN','')
final_data.Requisite = final_data.Requisite.str.replace('OR','/')
final_data.Requisite = final_data.Requisite.str.replace('PLAN','')

In [508]:
final_data.Requisite = final_data.Requisite.str.strip()

In [509]:
final_data.Requisite = final_data.Requisite.str.split('AND')

In [493]:
final_data.Requisite = final_data.Requisite.str.split('AND')

In [494]:
final_data.to_csv('Capstone_Data.csv',index=False)

In [510]:
final_data

,Subject_Catalog,Instructor_Name,Term_Offered,Descr,Long Title,Description,Label,Requisite
0,ACTG 2608,[Teruel Frank],[2 Units - Fall 2019],Financial Accounting,Financial Accounting,"This course introduces the roles, concepts, le...",Business Core,[]
1,MGMT 2501,"[Gordon Francine Ethel, Griffith Terri L, Zeit...","[3 Units - Fall 2018, 3 Units - Fall 2019, 3 U...",Bldg & Lding Hi-Perf Teams/Org,Building and Leading High-Performance Teams an...,Leaders are successful when they build teams a...,Business Core,[]
2,MSIS 2403,[Chavali Sreeram Kishore],[2 Units - Fall 2019],Database Management Systems -,Database Management Systems - Fundamentals of SQL,This course aims to give an understanding of a...,NaN,[]
3,MSIS 2406,[Chung Sunghun],"[2 Units - Fall 2019, 2 Units - Winter 2020]",R Programming,R Programming,Students will learn to manipulate and visualiz...,NaN,[]
4,MSIS 2407,"[Li Tao, Tan Tzu-Chin]",[4 Units - Winter 2020],Data Analytics - Python,Data Analytics - Python,Data analytics involves the application of sci...,NaN,[]
5,MSIS 2428,[Parsian Mahmoud],[4 Units - Spring 2020],Big Data Modeling & Analysis,Big Data Modeling & Analysis,Big Data and its role in carrying out modern b...,NaN,[]
6,MSIS 2429,[Khan Arshad],[2 Units - Spring 2020],Dashboards,Dashboards,This course enables you to transform data into...,NaN,[]
7,MSIS 2431,[Parsian Mahmoud],"[4 Units - Spring 2020, 4 Units - Winter 2020]",Machine Learning,Machine Learning,This course introduces participants to quantit...,NaN,[]
8,MSIS 2439,[Khan Arshad],[2 Units - Spring 2020],Data Visualization,Data Visualization,"\t\nThis course enables you to explore data, i...",NaN,[]
9,MSIS 2440,[Goldenberg Charles Bruce],[2 Units - Winter 2020],Practicum,Practicum,Practicum/Capstone course for MS Finance & Ana...,NaN,[FNCE 2400]
